In [1]:
import numpy as np

import seaborn as sns
import pandas as pd
import polars as pl
import os
import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import cvxpy as cp
import itertools
from scipy.special import logsumexp
from scipy.sparse import csr_matrix

sns.set(style='darkgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/vivarium-ecoli'))

from ecoli.processes.metabolism_redux import NetworkFlowModel, FlowResult

In [2]:
# Reading the matrix CSV files
def read_matrix(file_path, sparse=False):
    # add sparse matrix reading
    if sparse:
        return csr_matrix(pl.read_csv(file_path, has_header=False).to_numpy())
    else:
        return pl.read_csv(file_path, has_header=False).to_numpy()


# Import external data

In [3]:
# genes
gn = pl.read_csv("notebooks/fbagd/external_data/gene_protein.txt", separator='\t')

# reactions
rx = pl.read_csv("notebooks/fbagd/external_data/reactions.txt", separator='\t')

# ribosome profiling data
li = pl.read_csv("notebooks/fbagd/external_data/Li_2014_profiling.csv")

# C matrix as np sparse matrix
C = read_matrix("notebooks/fbagd/external_data/C_matrix.csv", sparse=True)

# join ribosome profiling data with genes on gene id
li = li.join(gn, left_on='Gene', right_on='Common-Name')
li.head()

In [4]:
rx.head()

# Get stoichiometric matrix

In [5]:
time = '10'
date = '2024-04-27'
experiment = 'convex_kinetics'
entry = f'{experiment}_{time}_{date}'
folder = f'out/cofactors/{entry}/'

In [6]:
output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

In [7]:
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

In [8]:
stoichiometry = agent['ecoli-metabolism-redux-classic'].stoichiometry
process_rxn_names = agent['ecoli-metabolism-redux-classic'].reaction_names
process_met_names = agent['ecoli-metabolism-redux-classic'].metabolite_names
process_rxn_enz_map = agent['ecoli-metabolism-redux-classic'].catalyzed_rxn_enzymes_idx
process_enz_names = agent['ecoli-metabolism-redux-classic'].catalyst_ids

# maintenance_reaction = agent['ecoli-metabolism-redux-classic'].model.maintenance_reaction
# stoichiometry["maintenance_reaction"] = maintenance_reaction

bad_rxns = ["RXN-12440", "TRANS-RXN-121", "TRANS-RXN-300", "TRANS-RXN-8", "R15-RXN-MET/CPD-479//CPD-479/MET.25.","DISULFOXRED-RXN[CCO-PERI-BAC]-MONOMER0-4152/MONOMER0-4438//MONOMER0-4438/MONOMER0-4152.71."]
# generate carbon mistake in parca, efflux/influx proton gen, iron cheating, mass gen
# for rxn in bad_rxns:
#    stoichiometry.pop(rxn, None)


In [23]:
# get flux example
fluxes_ar = np.array(fba['estimated_fluxes'][1:])
fluxes = pl.DataFrame(np.array(fba['estimated_fluxes'][1:]), schema=process_rxn_names)

flux_example = {rxn: flux for rxn, flux in zip(process_rxn_names, fluxes_ar[-1, :])}

# get nonzero flux rxns
nonzero_fluxes = {rxn: flux for rxn, flux in flux_example.items() if flux > 1e-6}

# filter S matrix to only include nonzero flux rxns
nz_rxn_idx = [process_rxn_names.index(rxn) for rxn in nonzero_fluxes.keys()]
stoich_nz = stoichiometry[:, nz_rxn_idx]

# remove zero columns
stoich_nz = stoich_nz[np.any(stoich_nz != 0, axis=1), :]

# export to csv, first matrix, then met names, then rxn names, then nonzero fluxes
folder = 'notebooks/fbagd/external_data/'
pl.DataFrame(stoich_nz).write_csv(folder + 'stoich_nz.csv', include_header=False)
pl.DataFrame(process_met_names).write_csv(folder + 'met_names.csv', include_header=False)
pl.DataFrame(process_rxn_names).write_csv(folder + 'rxn_names.csv', include_header=False)
pl.DataFrame(list(nonzero_fluxes.keys())).write_csv(folder + 'nonzero_rxn_fluxes.csv', include_header=False)


# Load enzyme counts

In [66]:
# from li, pick out mops minimal and product columns
 = li.select(["Product", "MOPS minimal"])

In [65]:
li

In [ ]:
# To start with, we will be simulating MOPS minimal. 